In [2]:
import tweepy
import re
from textblob import TextBlob
import pandas as pd
from pandas import DataFrame



import numpy as np
import joblib
import pickle
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score

In [3]:
api_key = 'FAQUNHC2FtBdXma5JwdbJsCt6'
api_secret_key = 'f8ZyKeNNuHoe6JvpPvYQD9keagttbjARQvaDrAqWfpVW28V4EY'
access_token = '1386283874901127168-2uJkEOsfczHSJB1tt598kBgvJ6ia5v'
access_token_secret = 'GYlCd3SnlTxVVfUWKZSjBGWFGuQ4mqngvMJRZvvu5Mxhg'

In [4]:
auth = tweepy.OAuthHandler(api_key,api_secret_key)
auth.set_access_token(access_token,access_token_secret)
api = tweepy.API(auth)

In [68]:
hasilSearch = api.search(q="vaksin",lang="en", count=1000)

In [69]:
hasil = []

In [70]:
for tweet in hasilSearch:
    tweet_properties = {}
    tweet_properties['tanggal_tweet'] = tweet.created_at
    tweet_properties['pengguna'] = tweet.user.screen_name
    tweet_properties['isi_tweet'] = tweet.text
    tweet_bersih = " ".join(re.sub("(@[A-Za-z0-9]+)|(^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",tweet.text).split())
    
    analysis = TextBlob(tweet_bersih)
    
    if analysis.sentiment.polarity > 0.0:
        tweet_properties["sentimen"] = "positif"
    elif analysis.sentiment.polarity == 0.0:
        tweet_properties["sentimen"] = "netral"
    else:
        tweet_properties["sentimen"] = "negatif"
    
    if tweet.retweet_count > 0:
        if tweet_properties not in hasil:
            hasil.append(tweet_properties)
    else:
        hasil.append(tweet_properties)
    
#     print(tweet_properties)

In [89]:
#hasil

In [72]:
df =  DataFrame(data = hasil)

In [73]:
df.head(5)

,tanggal_tweet,pengguna,isi_tweet,sentimen
0,2021-05-19 07:55:48,noorafieyda,RT @AmiKimmy: I had my first AZ jab today but ...,positif
1,2021-05-19 07:55:38,JapenKulim,INFO ON WHEELS PKP 3.0 2021 PEJABAT PENERANGAN...,netral
2,2021-05-19 07:54:57,nrlqstina,I’m so proud of everyone in my family. Semua e...,positif
3,2021-05-19 07:54:25,SyffaAzlan,@chrspepper Luckily the day after vaksin meman...,positif
4,2021-05-19 07:53:11,SafiuddinAhmad_,RT @mistaaimanvevo: @nadiazaman__ @DrDzul Acco...,netral


In [74]:
x = df['isi_tweet']
y = df['sentimen']

In [75]:
bow_transformer = CountVectorizer()
print(df['isi_tweet'].shape)
x = bow_transformer.fit_transform(df['isi_tweet'])

print(x.toarray())
print('shape of Sparse Matriks: ', x.shape)
print('Amount of Non-Zero occurrences: ', x.nnz)

filename1 = 'count_vectorized2.pkl'
pickle.dump(bow_transformer, open(filename1, 'wb'))

tf_transform = TfidfTransformer(use_idf=False).fit(x)
x = tf_transform.transform(x)
print(x.shape)

filename1 = 'tfid_transform2.pkl'
pickle.dump(tf_transform, open(filename1, 'wb'))

(100,)
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 1 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
shape of Sparse Matriks:  (100, 409)
Amount of Non-Zero occurrences:  2125
(100, 409)


In [76]:
density = (100.0 * x.nnz / (x.shape[0] * x.shape[1]))
print('Density: {}'.format((density)))

Density: 5.19559902200489


In [77]:
#splitting data
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2)


In [78]:
#Naive Bayes Method 
#classifier data
nb = MultinomialNB()
nb.fit(x_train, y_train)
preds = nb.predict(x_test)

In [79]:
print(classification_report(y_test, preds))

              precision    recall  f1-score   support

      netral       1.00      0.86      0.92         7
     positif       0.93      1.00      0.96        13

    accuracy                           0.95        20
   macro avg       0.96      0.93      0.94        20
weighted avg       0.95      0.95      0.95        20



In [80]:
from io import StringIO
classification = classification_report(y_test, preds)
s = StringIO(classification)
with open('classification.csv', 'w') as f:
    for line in s:
        f.write(line)

In [81]:
print (accuracy_score(y_test, preds))

0.95


In [82]:
accuracy = accuracy_score(y_test,preds)
a = np.asarray([accuracy])
np.savetxt('accuracy.csv', a, delimiter=',', fmt='%s')

In [83]:
Final = df[['tanggal_tweet','pengguna','isi_tweet','sentimen']]

In [84]:
Final.to_csv('hasil_analysis.csv')

In [85]:
model = LogisticRegression()
model.fit(x_train, y_train)

LogisticRegression()

In [86]:
filename = 'model_analisis.pkl'
pickle.dump(model, open(filename, 'wb'))

In [87]:
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(x_test, y_test)
print(result)

0.85


In [88]:
loaded_model.predict(x_test)

array(['netral', 'positif', 'positif', 'positif', 'positif', 'netral',
       'positif', 'netral', 'netral', 'netral', 'positif', 'positif',
       'positif', 'netral', 'netral', 'netral', 'netral', 'positif',
       'netral', 'positif'], dtype=object)